# Watershed With Seeds

In [74]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
road = cv2.imread('../DATA/road_image.jpg')

In [76]:
road_copy = np.copy(road) # or road.copy()

In [77]:
# We are not doing color correction because we only need open cv not matplotlib

In [78]:
road.shape

(600, 800, 3)

In [79]:
road.shape[:2]

(600, 800)

In [81]:
marker_image = np.zeros(road.shape[:2], dtype=np.int32) # get only the (x, y)

In [82]:
segments = np.zeros(road.shape, dtype=np.uint8)

In [83]:
segments.shape

(600, 800, 3)

In [84]:
marker_image.shape

(600, 800)

In [85]:
# color marpping
from matplotlib import cm

In [86]:
cm.tab10(0)

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)

In [87]:
cm.tab10(0)[:3] # get only first three

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765)

In [88]:
def create_rgb(i):
    return tuple(np.array(cm.tab10(i))[:3] * 255) # distict color 

In [89]:
colors = []
for i in range(10):
    colors.append(create_rgb(i))

In [90]:
colors

[(31.0, 119.0, 180.0),
 (255.0, 127.0, 14.0),
 (44.0, 160.0, 44.0),
 (214.0, 39.0, 40.0),
 (148.0, 103.0, 189.0),
 (140.0, 86.0, 75.0),
 (227.0, 119.0, 194.0),
 (127.0, 127.0, 127.0),
 (188.0, 189.0, 34.0),
 (23.0, 190.0, 207.0)]

In [91]:
###
# GLOBAL VARIABLES
# COLOR CHOICE
n_markers = 10 # 0 - 9
current_marker = 1
# Check markers updated by watershed
marks_updated = False

In [92]:
# CALLBACK FUNCTION
def mouse_callback(event, x, y , flags, param):
    global marks_updated
    
    if event == cv2.EVENT_LBUTTONDOWN:
        # markers passed to the watershed algo
        cv2.circle(marker_image, (x, y), 10, (current_marker), -1)
        
        # users sees on the road image
        cv2.circle(road_copy, (x, y), 10, colors[current_marker], -1)
        
        marks_updated = True

In [95]:
# WHILE TRUE
cv2.namedWindow('Road Image')
cv2.setMouseCallback('Road Image', mouse_callback)

while True:
    # SHOW 2 WINDOWS
    
    cv2.imshow('Watershed Segments', segments)
    cv2.imshow('Road Image', road_copy)
    
    # CLOSE ALL WINDOWS
    K = cv2.waitKey(1)

    if K == 27:
        break
    
    # CLEARING ALL THE COLORS PRESS C KEY
    elif K == ord('c'):
        # RESETS
        road_copy = road.copy()
        marker_image = np.zeros(road.shape[:2], dtype=np.int32)
        segments = np.zeros(road.shape, dtype=np.uint8)
    # UPDATE COLOR CHOICE
    elif K > 0 and chr(K).isdigit():
        current_marker = int(chr(K))
    
    # UPDATE THE MARKINGS
    if marks_updated :
        marker_image_copy = marker_image.copy()
        cv2.watershed(road, marker_image_copy)
        
        segments = np.zeros(road.shape, dtype=np.uint8)
        
        for color_index in range(n_markers):
            # COLORING THE SEGMENTS, NUMPY CALL
            segments[marker_image_copy == (color_index)] = colors[color_index]
    
cv2.destroyAllWindows()